# Diceloader Triage Notes
> Researching malware downloaders, detection and triage

- toc: true 
- badges: true
- categories: [downloader,diceloader,yara,triage]

## Overview

According to Mandiant..
> the DiceLoader framework, a known toolkit that helps attackers gain a foothold in infected systems and perform reconnaissance


According to Twitter...

> twitter: https://twitter.com/c3rb3ru5d3d53c/status/1348667319665487874

and ... 

> twitter: https://twitter.com/Arkbird_SOLG/status/1310966874352635907

and ...

> twitter: https://twitter.com/MrDanPerez/status/1347185968500109313


According to Dan Perez at Mandiant...

>For those tracking #FIN7 - #TAKEOUT is the powershell memory dropper that has been seen dropping #CARBANAK OR #DICELOADER - there's some confusion going on..
ClearTemp.ps1 files == TAKEOUT
Payloads embedded == CARBANAK || DICELOADER


### References 
- [MalwareBazaar "diceloader" tag](https://bazaar.abuse.ch/browse.php?search=tag%3Adiceloader)
- [Attributed to GOLD NIAGARA (carbon spider)](https://www.secureworks.com/research/threat-profiles/gold-niagara)
- [Attributed to FIN7](https://duo.com/decipher/fin7-evolves-with-novel-malware-initial-access-tactics)
- [Possible that Lizar is an earlier version of Diceloader](https://bi-zone.medium.com/from-pentest-to-apt-attack-cybercriminal-group-fin7-disguises-its-malware-as-an-ethical-hackers-c23c9a75e319)

## Triage Notes

**Sample:** `2d88767c424d05330839e568b32f9f52962df56b1d3021f69930167fe623efd1`

**Creation Time:**  2021-07-15 15:46:07 UTC

**First Submission:** 2022-03-20 23:22:32 UTC

- The sample is an x64 DLL with two exports
- One of the exports is a reflective PE loader (possibly copy-paste from metasploit)
- There is a mutex string `Global\\%08x` that is built as a stack string

```
.text:0000000180001749 C7 44 24 30 47 6C 6F 62                 mov     dword ptr [rsp+228h+var_mutex], 626F6C47h
.text:0000000180001751 C7 44 24 34 61 6C 5C 25                 mov     dword ptr [rsp+228h+var_mutex+4], 255C6C61h
.text:0000000180001759 C7 44 24 38 30 38 78 00                 mov     dword ptr [rsp+228h+var_mutex+8], 783830h
```

- The `.data` section containst two encrypted blobs with a 32-byte key sandwitched between them. 
- The first encrypted blob seems to contain a port number

```
01 bb 01 11 bb 01 00 00 00 90 f1 
```

- The second encrypted blob contains C2 IP addresses separated with a `|`

```
46.17.107.7|185.250.151.33\x00
```



## Yara

```
import "pe"

rule diceloader {
    meta:
        description = "Identifies diceloader"
        
   strings:
       
       // gobal stack string
       // C7 44 24 30 47 6C 6F 62                 mov     dword ptr [rsp+228h+var_mutex], 626F6C47h
       // C7 44 24 34 61 6C 5C 25                 mov     dword ptr [rsp+228h+var_mutex+4], 255C6C61h
       // C7 44 24 38 30 38 78 00                 mov     dword ptr [rsp+228h+var_mutex+8], 783830h
       $x1 = { C7 ?? ?? ?? 47 6C 6F 62 C7 ?? ?? ?? 61 6C 5C 25 C7 ?? ?? ?? 30 38 78 00 }
       
       // fnv1
       // 48 FF C1                                inc     rcx
       // 33 C2                                   xor     eax, edx
       // 69 C0 93 01 00 01                       imul    eax, 1000193h
       $x2 = { 48 FF C1 33 C2 69 C0 93 01 00 01 }
       
       // sleep(1000)
       // B9 10 27 00 00                          mov     ecx, 2710h      ; dwMilliseconds
       // FF 15 B3 13 00 00                       call    cs:Sleep
       $x3 = {B9 10 27 00 00 ff}
       
       // hash for NtFlushInstructionCache
       // 81 F9 B8 0A 4C 53                       cmp     ecx, 534C0AB8h
       // 75 16                                   jnz     short loc_7FFA3BA4121E
       $x4 = { 81 F9 B8 0A 4C 53 75 }
       
   condition:
       pe.is_64bit() and 
       all of ($x*)

}
```


## Config Extraction

We want to pull out the C2 IP addresses.

In [10]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

In [6]:
import pefile
import struct
import re


FILE_PATH = '/tmp/diceloader/2d88767c424d05330839e568b32f9f52962df56b1d3021f69930167fe623efd1.exe'
file_data = open(FILE_PATH, 'rb').read()

In [32]:
def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i%len(key)])
    return bytes(out)

pe = pefile.PE(data=file_data)


data_section_offset= None
data_section_end_offset = None


for s in pe.sections:
    if b'.data\x00' in s.Name:
        data_section_offset = s.PointerToRawData
        data_section_end_offset = s.PointerToRawData + s.SizeOfRawData
        break


# Find the key based on it being passed as an arg
#
# BD 1F 00 00 00                          mov     ebp, 1Fh
# 4C 8D 05 68 1E 00 00                    lea     r8, key
# 44 8B CD                                mov     r9d, ebp
# 49 8B CE                                mov     rcx, r14
# 8D 75 F7                                lea     esi, [rbp-9]
# 8B D6                                   mov     edx, esi
# E8 50 FE FF FF                          call    sub_7FFA3BA420B8
egg = rb'\xBD\x1F\x00\x00\x00\x4C\x8D\x05(....)'

key = None

for m in re.finditer(egg, file_data):
    rel_offset  = struct.unpack('<I',m.group(1))[0] 
    # Remember to add 7 for the 7 bytes in the instruction
    rel_offset += 7 
    print(f"rel offset: {hex(rel_offset)}")
    # Remember that this is in the .text section which is loaded before the .data 
    # so the relative will be postive displacement
    match_offset = m.start() + 5
    print(f"match offset: {hex(match_offset)}")
    match_rva = pe.get_rva_from_offset(match_offset)
    print(f"match rva: {hex(match_rva)}")
    key_rva = match_rva + rel_offset
    print(f"key rva: {hex(key_rva)}")
    key_offset = pe.get_offset_from_rva(key_rva)
    print(f"key offset: {hex(key_offset)}")
    key = file_data[key_offset:key_offset+31]
    print(tohex(key))

# Use the key position to locate the other two data blobs

blob1 = file_data[data_section_offset:key_offset]
blob2 = file_data[key_offset+31:data_section_end_offset]


out_blob1 = xor_decrypt(blob1, key)
port = struct.unpack('>H',out_blob1[:2])[0]
print(f"Port: {port}")

out_blob2 = xor_decrypt(blob2.lstrip(b'\x00'), key)

c2_ips  = out_blob2.split(b'\x00')[0].split(b'|')
for c2 in c2_ips:
    print(f"c2: {c2}")



rel offset: 0x1e6f
match offset: 0x1651
match rva: 0x2251
key rva: 0x40c0
key offset: 0x2ec0
b'b2f3a1eaabea5757c9d7411c0df59d4ea951440911ec7d6603253e7d10660a'
Port: 443
c2: b'46.17.107.7'
c2: b'185.250.151.33'
